In [2]:
!pip install -q google-generativeai requests beautifulsoup4 ipywidgets

import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ✅ Configure Gemini API
import os
from dotenv import load_dotenv

# Load variables from .env
load_dotenv()

# Get API key securely
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("⚠ GEMINI_API_KEY not found in environment variables!")

genai.configure(api_key=api_key)  # <-- Add this line

# ✅ Function to extract clean text from website
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Handle HTTP errors
        soup = BeautifulSoup(response.content, 'html.parser')
        for tag in soup(["script", "style"]):
            tag.decompose()
        return ' '.join(soup.stripped_strings)
    except Exception as e:
        return f"❌ Error fetching content: {str(e)}"

# ✅ Summarize using Gemini Flash
def summarize_with_gemini(text):
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")  # Correct model name
        response = model.generate_content(
            f"You are a helpful summarizer. Summarize this website content in bullet points:\n\n{text[:8000]}"
        )
        return response.text
    except Exception as e:
        return f"⚠ Gemini API Error: {str(e)}"

# ✅ UI Widgets
url_input = widgets.Text(
    value='',
    placeholder='Enter website URL...',
    description='🌐 URL:',
    layout=widgets.Layout(width='100%')
)

output = widgets.Output()

# ✅ Button click handler
def on_click(b):
    with output:
        clear_output()
        print("🔄 Fetching and summarizing, please wait...")
        url = url_input.value.strip()
        if not url.startswith("http"):
            url = "http://" + url

        text = extract_text_from_url(url)
        if text.startswith("❌ Error"):
            print(text)
            return

        summary = summarize_with_gemini(text)
        display(HTML(f"<h4>📄 Website Summary:</h4><p>{summary.replace(chr(10), '<br>')}</p>"))

# ✅ Display button
button = widgets.Button(description="Summarize with Gemini", button_style='success')
button.on_click(on_click)

# ✅ Display UI
display(url_input, button, output)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Text(value='', description='🌐 URL:', layout=Layout(width='100%'), placeholder='Enter website URL...')

Button(button_style='success', description='Summarize with Gemini', style=ButtonStyle())

Output()